In [ ]:
import numpy as np
import sys
import os
import plotly.graph_objects as go
import plotly.express as px
from toolkit.common.constants import *
from toolkit.cars.car_configuration import Car
from toolkit.lap.gps import *
from toolkit.mmd import MMD
from toolkit.steady_state_solver.ls_optimize import LS_Solver
from toolkit.steady_state_solver.iterative import Iterative_Solver
from toolkit.steady_state_solver.parachute import Parachute
from toolkit.common.maths import interpolate, sa_lut

import scipy
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [ ]:
long_g = 1.2
lat_g = 2.0
brake_g = -1.7
ax = long_g * 9.81
ay = lat_g * 9.81
axb = brake_g * 9.81

ADF = 0.0774
ASR = 0.2378

In [ ]:
car = Car()
# Iyy = car.izz * (car.mass_sprung / car.mass_unsprung)**2
pch = 1.68*IN_TO_M
Iyy = 50 + (car.mass_sprung * pch**2)
Izz = car.izz
Ixx = 7.5 + ( car.mass_sprung * ((car.z_f + car.z_r)*0.5)**2)


m_tire = car.mass_unsprung / 4
k_tire = 650 * LB_PER_IN_TO_N_PER_M

_,_,_,_, wt_pitch, _ = car.find_contact_patch_loads(long_g = ax, lat_g = 0, vel = 10, used_aero = True)
_,_,_,_, wt_brake, _ = car.find_contact_patch_loads(long_g = axb, lat_g = 0, vel = 20, used_aero = True)
_,_,_,_,_, wt_roll = car.find_contact_patch_loads(long_g = 0, lat_g = ay, vel = 20, used_aero = True)


print(wt_pitch)
print(wt_brake)
print(wt_roll)
print(223.11 * LB_PER_IN_TO_N_PER_M)
print(18242*2)
# k = car.k_r_b

print(car.mass_sprung/2)
print(car.mass_sprung*car.front_axle_weight)
print(car.mass_sprung*(1-car.front_axle_weight))
print(car.mass_unsprung/4)

print("REAR TRAVEL")
print((wt_pitch / car.k_r_b)/IN_TO_M)
# print(((ax*car.mass_sprung*car.cg_height)/car.k_r_b)/IN_TO_M)

In [ ]:
mode = 'Brake' # Roll, Brake, or Drive
sim = 'Phase Damped' # Undamped, Damped, or Phase Damped (Case Sensitive)
phase_delay = 0.1 # 0.1, rec 0.2 if roll
Ccrit = 0.65 # Damping ratio


T_y_a = wt_pitch * car.b
T_y_b = wt_brake * car.a
T_x = wt_roll * ((car.front_track + car.rear_track)*0.5)

if mode == 'Drive':
    # Bump
    T = T_y_a
    I = Iyy
    L = car.b
    # k = car.k_r_b
    k = 150 / (1.1**2) * LB_PER_IN_TO_N_PER_M
elif mode == 'Roll': 
    # Roll
    T = T_x
    I = Ixx
    L = car.rear_track / 2
    # 128-240 lb/in, pick 2. MUST BE EFFECTIVE SINGLE WHEEL BUMP RATE IN ROLL W ARB
    k = (128+140) * LB_PER_IN_TO_N_PER_M
else:
    # Bump
    T = T_y_b
    I = Iyy
    L = car.a
    # k = car.k_f_b
    k = 150 / (1.1**2) * LB_PER_IN_TO_N_PER_M
    

c = 1126 * 2 * Ccrit
c_imp = c / LB_PER_IN_TO_N_PER_M

print(f"Applying {c//2} N/(m/s) Damping Rate Per Shock")
print(f"Applying {c_imp/2} lb/(in/s) Damping Rate Per Shock")
print(f"Expected Rate at 5in/s: {c_imp*5/2}lb/(in/s)")
print(f"Expected Rate at 8in/s: {c_imp*8/2}lb/(in/s)")

def phase_input(Tapp, t):
    if t > phase_delay:
        return Tapp
    else:
        return Tapp * t / phase_delay

# I_yy_pc = 50
# I_yy_pc = car.mass_sprung * pch**2
# Define the system of differential equations without the damper
def undamped_beam_dynamics(t, y):
    theta, omega = y  # y[0] = theta, y[1] = omega (angular velocity)
    dtheta_dt = omega
    domega_dt = (T - k * L**2 * theta) / I
    return [dtheta_dt, domega_dt]

def beam_dynamics(t, y):
    theta, omega = y  # y[0] = theta, y[1] = omega (angular velocity)
    dtheta_dt = omega
    domega_dt = (T - c * L**2 * omega - k * L**2 * theta) / I
    return [dtheta_dt, domega_dt]

def phased_beam_dynamics(t, y):
    theta, omega = y
    dtheta_dt = omega
    domega_dt = (phase_input(T, t) - c * L**2 * omega - k * L**2 * theta) / I
    return [dtheta_dt, domega_dt]

# Solve the ODEs for the undamped system
theta_0 = 0
omega_0 = 0
# Time span for simulation
t_span = (0, 1)  # Simulate from t=0 to t=10 seconds
t_eval = np.linspace(t_span[0], t_span[1], 500)  # Points to evaluate

if sim == 'Damped':
    solution_undamped = solve_ivp(beam_dynamics, t_span, [theta_0, omega_0], t_eval=t_eval)
elif sim == 'Unamped':
    solution_undamped = solve_ivp(undamped_beam_dynamics, t_span, [theta_0, omega_0], t_eval=t_eval)
else: 
    solution_undamped = solve_ivp(phased_beam_dynamics, t_span, [theta_0, omega_0], t_eval=t_eval)

# Extract results
time_undamped = solution_undamped.t
theta_undamped = solution_undamped.y[0] * L / IN_TO_M  # Linear displacement
omega_undamped = solution_undamped.y[1] * L / IN_TO_M # Linear velocity

# Plot results for the undamped system
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time_undamped, theta_undamped, label=r'z(t) (in)')
plt.title(f'Displacement and Velocity of Shocks in {mode} ({sim})')
plt.xlabel('Time (s)')
plt.ylabel('Displacement (in)')
plt.grid()
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(time_undamped, omega_undamped, label=r'v(t) (in/s)', color='orange')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (in/s)')
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
mode = 'Brake' # Roll, Brake, or Drive
sim = 'Phase Damped' # Damped or Phase Damped (Case Sensitive)
phase_delay = 0.1 # 0.1, rec 0.2 if roll
Ccrit = 0.65 # Damping ratio


T_y_a = wt_pitch * car.b
T_y_b = wt_brake * car.a
T_x = wt_roll * ((car.front_track + car.rear_track)*0.5)

if mode == 'Drive':
    # Bump
    T = T_y_a
    I = Iyy
    L = car.b
    # k = car.k_r_b
    k = 150 / (1.1**2) * LB_PER_IN_TO_N_PER_M
elif mode == 'Roll': 
    # Roll
    T = T_x
    I = Ixx
    L = car.rear_track / 2
    # 128-240 lb/in, pick 2. MUST BE EFFECTIVE SINGLE WHEEL BUMP RATE IN ROLL W ARB
    k = (128+140) * LB_PER_IN_TO_N_PER_M
else:
    # Bump
    T = T_y_b
    I = Iyy
    L = car.a
    # k = car.k_f_b
    k = (150 / (1.1*1.1)) * LB_PER_IN_TO_N_PER_M

print(T)
print(I)
print(L)
print(k)
print("temp")
print(car.k_r_b)
print(k - car.k_r_b)
print(k_tire)
print(m_tire)

c = 1126 * 2 * Ccrit
c_imp = c / LB_PER_IN_TO_N_PER_M

print(f"Applying {c//2} N/(m/s) Damping Rate Per Shock")
print(f"Applying {c_imp/2} lb/(in/s) Damping Rate Per Shock")
print(f"Expected Rate at 5in/s: {c_imp*5}lb/(in/s)")
print(f"Expected Rate at 8in/s: {c_imp*8}lb/(in/s)")

def T_app(t):
    if t > phase_delay:
        return T
    else:
        return T * t / phase_delay


# Define the system of equations
def system(t, y):
    theta, omega, x1, v1, x2, v2 = y

    # Applied torque at time t
    torque = T_app(t)
    
    # Spring-damper force at the end of the beam
    F_spring_damper = -k * x1 - c * (v2 - L * omega)
    
    # Ground spring force acting on the tire mass
    F_ground = -k_tire * x2
    
    # Equations of motion
    dtheta_dt = omega
    domega_dt = (1 / I) * (torque - F_spring_damper * L)
    dx1_dt = v1
    dv1_dt = (1 / m_tire) * (F_spring_damper)
    dx2_dt = v2
    dv2_dt = (1 / m_tire) * (F_ground + F_spring_damper)

    return [dtheta_dt, domega_dt, dx1_dt, dv1_dt, dx2_dt, dv2_dt]

# Initial conditions: [theta, omega, x1, v1, x2, v2]
y0 = [0.0, 0.0, 0.1, 0.0, 0.0, 0.0]
t_span = (0, 1)  # Time span for the simulation
t_eval = np.linspace(0, max(t_span), 5000)  # Time points at which to evaluate the solution

# Solve the system
sol = solve_ivp(system, t_span, y0, t_eval=t_eval, method='RK45')

# Extract the velocity of the damper (relative velocity between the tire mass and the beam's end)
damper_velocity = sol.y[4] - L * sol.y[1]  # Relative velocity between x2 (tire mass) and L * omega (beam's end)

# Force between m_tire and the ground (from the ground spring)
F_ground = -k_tire * sol.y[4]

# Plot the results
plt.figure(figsize=(10, 6))

# Plot the velocity of the damper (relative velocity)
plt.subplot(2, 1, 1)
plt.plot(sol.t, damper_velocity, label="Damper Compression Velocity", color="blue")
plt.xlabel("Time (s)")
plt.ylabel("Velocity (m/s)")
plt.title("Damper Compression Velocity")
plt.grid(True)

# Plot the force between m_tire and the ground
plt.subplot(2, 1, 2)
plt.plot(sol.t, F_ground, label="Force between m_tire and Ground", color="red")
plt.xlabel("Time (s)")
plt.ylabel("Force (N)")
plt.title("Force between m_tire and Ground")
plt.grid(True)

plt.tight_layout()
plt.show()
# # Solve the ODEs for the undamped system
# theta_0 = 0
# omega_0 = 0
# # Time span for simulation
# t_span = (0, 1)  # Simulate from t=0 to t=10 seconds
# t_eval = np.linspace(t_span[0], t_span[1], 500)  # Points to evaluate

# if sim == 'Damped':
#     solution_undamped = solve_ivp(beam_dynamics, t_span, [theta_0, omega_0], t_eval=t_eval)
# elif sim == 'Unamped':
#     solution_undamped = solve_ivp(undamped_beam_dynamics, t_span, [theta_0, omega_0], t_eval=t_eval)
# else: 
#     solution_undamped = solve_ivp(phased_beam_dynamics, t_span, [theta_0, omega_0], t_eval=t_eval)

# # Extract results
# time_undamped = solution_undamped.t
# theta_undamped = solution_undamped.y[0] * L / IN_TO_M  # Linear displacement
# omega_undamped = solution_undamped.y[1] * L / IN_TO_M # Linear velocity

# # Plot results for the undamped system
# plt.figure(figsize=(12, 6))
# plt.subplot(2, 1, 1)
# plt.plot(time_undamped, theta_undamped, label=r'z(t) (in)')
# plt.title(f'Displacement and Velocity of Shocks in {mode} ({sim})')
# plt.xlabel('Time (s)')
# plt.ylabel('Displacement (in)')
# plt.grid()
# plt.legend()

# plt.subplot(2, 1, 2)
# plt.plot(time_undamped, omega_undamped, label=r'v(t) (in/s)', color='orange')
# plt.xlabel('Time (s)')
# plt.ylabel('Velocity (in/s)')
# plt.grid()
# plt.legend()

# plt.tight_layout()
# plt.show()
